In [25]:
import matplotlib.pyplot as plt
import random

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

In [26]:
df = pd.read_csv("../../data/image_classification.csv")

x = df.iloc[:, :-1]
y = pd.get_dummies(df.iloc[:, -1]).astype(int)

x_ssc = StandardScaler()
x_scaled = x_ssc.fit_transform(x)

x_tensor = torch.tensor(x_scaled,dtype=torch.float32)
y_tensor = torch.tensor(y.values,dtype=torch.float32)

x_train, x_test, y_train, y_test = train_test_split(x_tensor, y_tensor, test_size=0.2, shuffle=True)

In [27]:
USE_mps = torch.backends.mps.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴

device = torch.device("mps" if USE_mps else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
print("다음 기기로 학습합니다:", device)

다음 기기로 학습합니다: mps


In [28]:
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, drop_last=True)

linear = nn.Linear(64,10,bias=True).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

In [29]:
for epoch in range(15): # 앞서 training_epochs의 값은 15로 지정함.
    avg_cost = 0
    total_batch = len(dataloader)

    for X, Y in dataloader:
        # 배치 크기가 100이므로 아래의 연산에서 X는 (100, 784)의 텐서가 된다.
        X = X.to(device)
        # 레이블은 원-핫 인코딩이 된 상태가 아니라 0 ~ 9의 정수.
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.977361977
Epoch: 0002 cost = 0.375237226
Epoch: 0003 cost = 0.275351912
Epoch: 0004 cost = 0.229516774
Epoch: 0005 cost = 0.200820684
Epoch: 0006 cost = 0.175513715
Epoch: 0007 cost = 0.166485265
Epoch: 0008 cost = 0.155506805
Epoch: 0009 cost = 0.145535916
Epoch: 0010 cost = 0.134271920
Epoch: 0011 cost = 0.128250465
Epoch: 0012 cost = 0.124002397
Epoch: 0013 cost = 0.118416160
Epoch: 0014 cost = 0.114657871
Epoch: 0015 cost = 0.109965019
Learning finished


In [ ]:
accuracy_score(y_test.argmax(axis=1).cpu().detach().numpy(), linear(x_test).argmax(axis=1).cpu().detach().numpy())

0.9805555555555555